## Task 2 -- Web scraper + RAG Agent

In [5]:
%pip install -qU langchain langchain-huggingface langchain-cohere sentence-transformers langgraph "langchain[google-genai]" langchain_tavily langchain_community

Note: you may need to restart the kernel to use updated packages.


In [6]:
from __future__ import annotations
import os, time, requests
from typing import List, Dict, Optional

from bs4 import BeautifulSoup
from pydantic import BaseModel

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document

from langchain_cohere import ChatCohere

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]
TAVILY_API_KEY = os.environ["TAVILY_API_KEY"]
SERPAPI_KEY = os.environ["SERPAPI_KEY"]

from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY, 
    temperature=0
)


In [8]:
# 1. Tavily Agent
from langchain_tavily import TavilySearch

tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
    api_key=TAVILY_API_KEY
)

tavily_agent = create_react_agent(llm, [tavily_search_tool])

# 2. SerpAPI Agent
from langchain_community.utilities import SerpAPIWrapper
from langchain_core.tools import Tool

serp = SerpAPIWrapper(
    serpapi_api_key=SERPAPI_KEY
)
serp_tool = Tool(
    name="serpapi-search",
    func=serp.run,
    description="Search engine powered by SerpAPI"
)

serp_agent = create_react_agent(llm, [serp_tool])

# 3. DuckDuckGO Agent
from langchain_community.tools import DuckDuckGoSearchRun

duckduckgo_tool = DuckDuckGoSearchRun()

ddg_agent = create_react_agent(llm, [duckduckgo_tool])


# Combined Agent
agent = create_react_agent(llm, [tavily_search_tool, duckduckgo_tool, serp_tool])


In [10]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA
GOOGLE_API_KEY = os.environ["GEMINI_API_KEY"]

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [11]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

def scrape_with_rag(urls, query: str):
    """Scrape URLs, build FAISS index, run RAG for query."""
    docs = []
    for url in urls:
        try:
            loader = WebBaseLoader(url)
            docs.extend(loader.load())
        except Exception as e:
            print(f"Failed to scrape {url}: {e}")

    if not docs:
        return " No content scraped."

    # Chunk text
    splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=150)
    splits = splitter.split_documents(docs)

    # Temporary FAISS index
    db = FAISS.from_documents(splits, embeddings)
    retriever = db.as_retriever(search_kwargs={"k": 4})

    # RetrievalQA pipeline
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff"
    )
    return qa_chain.run(query)


In [12]:
def safe_query(user_input: str, use_rag=False):
    """Query multi-agent with optional RAG scraping fallback."""
    print(f"\n## Query: {user_input}\n")

    try:
        urls = []  # Collect URLs mentioned
        answer = None

        # Step 1: Run search
        for step in agent.stream({"messages": user_input}, stream_mode="values"):
            msg = step["messages"][-1]
            msg.pretty_print()

            if "http" in msg.content:
                urls.extend([word for word in msg.content.split() if word.startswith("http")])

        # Step 2: Decide if RAG is needed
        if use_rag and urls:
            print("\n### Using RAG on scraped sites...\n")
            answer = scrape_with_rag(urls[:5], user_input)
            print(answer)

    except Exception as e:
        print(f"\n**Primary agent failed:** {e}\n")
        try:
            print("### Fallback: DuckDuckGo Search\n")
            print(duckduckgo_tool.run(user_input))
        except Exception as e2:
            print(f"**All search methods failed:** {e2}")

In [14]:
# Normal search only
safe_query("What is the difference between LangChain and LlamaIndex?")

# Search + RAG scraping
safe_query("Summarize LangChain vs LlamaIndex from official docs.", use_rag=True)



## Query: What is the difference between LangChain and LlamaIndex?

================================ Human Message =================================

What is the difference between LangChain and LlamaIndex?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (f4cc91d6-e0c4-4803-9eac-5f7c97a50b05)
 Call ID: f4cc91d6-e0c4-4803-9eac-5f7c97a50b05
  Args:
    query: difference between LangChain and LlamaIndex
================================= Tool Message =================================
Name: tavily_search

{"query": "difference between LangChain and LlamaIndex", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://stackoverflow.com/questions/76990736/differences-between-langchain-llamaindex", "title": "Differences between Langchain & LlamaIndex [closed]", "content": "You'll be fine with just LangChain, however, LlamaIndex is optimized for indexing, and retrieving data. Retrieval-Augmented Generat

In [18]:
safe_query("Explain the working of a transformer", use_rag=False)


## Query: Explain the working of a transformer

================================ Human Message =================================

Explain the working of a transformer
================================== Ai Message ==================================
Tool Calls:
  tavily_search (480e13fb-e66c-4e5c-ac65-80562a761a0c)
 Call ID: 480e13fb-e66c-4e5c-ac65-80562a761a0c
  Args:
    query: how does a transformer model work
================================= Tool Message =================================
Name: tavily_search

{"query": "how does a transformer model work", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://blogs.nvidia.com/blog/what-is-a-transformer-model/", "title": "What Is a Transformer Model? | NVIDIA Blogs", "content": "# What Is a Transformer Model? transformer model A transformer model is a neural network that learns context and thus meaning by tracking relationships in sequential data like the words in this sentence. ## **What Can Transfor

In [17]:
safe_query("Explain the working of a transformer", use_rag=True)


## Query: Explain the working of a transformer

================================ Human Message =================================

Explain the working of a transformer
================================== Ai Message ==================================
Tool Calls:
  tavily_search (c4e0c07f-afea-4595-a60c-8d47a9e41286)
 Call ID: c4e0c07f-afea-4595-a60c-8d47a9e41286
  Args:
    query: how does a transformer model work
================================= Tool Message =================================
Name: tavily_search

{"query": "how does a transformer model work", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://blogs.nvidia.com/blog/what-is-a-transformer-model/", "title": "What Is a Transformer Model? | NVIDIA Blogs", "content": "# What Is a Transformer Model? transformer model A transformer model is a neural network that learns context and thus meaning by tracking relationships in sequential data like the words in this sentence. ## **What Can Transfor